In [ ]:
#import custom class
from defs import KernelApprox

#importing libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sympy 

import time
from tqdm.notebook import tqdm 

from itertools import chain, combinations #import function for powerset

import concurrent.futures #import function for parallelization
import multiprocessing #for getting number of cores

In [ ]:
#defining functions
def f(y): 
    fy = y**4 - 2*y**3 + y**2 -1/30
    try:
        d = y.shape[1]
        return np.prod(1 + (fy/(np.arange(1,d+1)**4)), axis=1, keepdims=True)
    except: #1d
        return 1 + fy

def g(y, lam = 0.01):
    return abs(np.cos(lam*2*np.pi*y + 0.5*np.pi) )/( lam*2*np.pi )


#function for generating samples
def gen_samples(f, K, d, const = 1.1):
    extra = const**2 + 0.05 + 1/np.sqrt(K*0.1) #tends to const**2 + 0.05 as K -> inf
    K_adj = int(K*extra)

    u = scipy.stats.uniform.rvs(size = K_adj)
    y = scipy.stats.uniform.rvs(size = (K_adj, d))

    fy = f(y)

    Y_est = y[np.where(np.less(u,fy[:,0]/(const**2)))]
    return Y_est[:K]

# Example use of class

In [ ]:
d = 1 #dimension
alpha = 4 #smoothing parameter

M = int(1e7) #number of samples
N = 11 #number of points to evaluate at (should be prime). If N not given, will be chosen automatically based on M
lam = 0.01 #regularisation parameter. If lam not given, will be chosen automatically based on M

#number of cores to use for parallelization
num_workers = 12 #if not given, will be automatically chosen (will not use more than N cores)

#generate samples
Y = gen_samples(f, M, d)

#create class instance
approx = KernelApprox(Y, lamda = lam, n = N, alpha = 4, num_workers = 12, verbose = True)

approx.gen_c() #generate c

In [ ]:
y = np.array([np.linspace(0,1,1000)]).T #points to evaluate at
approx.kern_est(y) #estimate kernel

In [ ]:
#estimate on a grid with 100 points
approx.kern_est_grid(100)

approx.plot_est() #plots estimate of kern_est_grid in 1d or 2d (will run kern_est_grid if not already run)

In [ ]:
#estimate MISE
approx.calc_error_L2(f) #calculate L2 error

#approx.L2_error #error estimate
#approx.L2_error_var #variance of error estimate

In [ ]:
#estimate weak error
approx.calc_error(f, g, K = int(1e7)) #calculate weak error

#approx.error #error estimate
#approx.error_var #variance of error estimate

# Code for looking at error convergence

## N & M

In [ ]:
d = 6
alpha = 4

#number of workers for parallel processing
num_workers = 12

#number of times to compute the error for each size
num_intervals = 100

#constant for sampling
samp_const = 1.2  #(should be > f_max)

#values to iterate over
size = np.arange(1,7+1) # M = 10**size is the number of samples
K_size = 7 # K = 10**K_size is the number of samples used to compute the kernel estimate
num_points =  np.array([2,3,5,7,11,13,17,19,23]) #number of lattice points

alpha_text = ""
if alpha != 4:
    alpha_text = "".join([str(alpha), "_", alpha_text])

#vector for storing the errors (means)
mean_err = np.zeros((len(num_points), len(size))).T
mean_err_var = np.zeros((len(num_points), len(size))).T
mean_err_M_var = np.zeros((len(num_points), len(size))).T
mean_L2_err = np.zeros((len(num_points), len(size))).T
mean_L2_err_var = np.zeros((len(num_points), len(size))).T
mean_L2_err_M_var = np.zeros((len(num_points), len(size))).T
mean_sum_c = np.zeros((len(num_points), len(size))).T


#lattice for L2 error
Y = gen_samples(f, int(1e2), d)
approx = KernelApprox(Y, 0.01, n = int(1e4), alpha = alpha, num_workers = num_workers)
approx.gen_lattice()
lattice = approx.lattice

for j in tqdm(range(len(size))): #iterate over the number of samples (M)
    print('='*80) #for clarity (splits iterations)
    print("M = 10^", size[j])
    M = int(10**size[j]) #number of samples

    #adjust based on M to get good estimates of the error
    num_intervals_adjusted = num_intervals #* max( 5 - j , 1)

    #vector for storing the errors
    err = np.zeros((len(num_points), num_intervals_adjusted))
    err_var = np.zeros((len(num_points), num_intervals_adjusted))
    L2_err = np.zeros((len(num_points), num_intervals_adjusted))
    L2_err_var = np.zeros((len(num_points), num_intervals_adjusted))
    sum_c = np.zeros((len(num_points), num_intervals_adjusted))

    for num in tqdm(range(num_intervals_adjusted)): #iterate over the number of times to compute the error for each size


        #generate the samples
        Y = gen_samples(f, M, d)

        for i in tqdm(range(len(num_points)), leave = False, disable=True): #iterate over the number of lattice points (n)
        
            #####################################
                
            #compute the kernel estimate
            approx = KernelApprox(Y, 0.01, n = num_points[i], alpha = alpha, num_workers = num_workers, verbose=False)
            approx.gen_c()

            np.save("".join(["c_", str(d), "d.npy"]), approx.c) #SAVE THE C VALUES

            ##################################
            K = int(10**K_size) #number of samples used to compute the kernel estimate (K)
            
            err[i, num] = approx.calc_error(f, g, K, num_workers = num_workers, const = samp_const)
            err_var[i, num] = approx.error_var

            L2_err[i, num] = approx.calc_error_L2(f, N = int(1e4), x = lattice)
            L2_err_var[i, num] = approx.L2_error_var

            sum_c[i, num] = sum(approx.c)
            
            mean_err[j] = np.mean(  ( err ), axis = 1)
            mean_err_var[j] = np.mean(  ( err_var ), axis = 1)
            mean_err_M_var[j] = np.var(  ( err ), axis = 1)
            mean_L2_err[j] = np.mean(  ( L2_err ), axis = 1)
            mean_L2_err_var[j] = np.mean(  ( L2_err_var ), axis = 1)
            mean_L2_err_M_var[j] = np.var(  ( L2_err ), axis = 1)
            mean_sum_c[j] = np.mean(  ( sum_c ), axis = 1)

            np.save("".join(["NM_", alpha_text, "error_", str(d), "d.npy"]), mean_err) #SAVE THE ERRORS
            np.save("".join(["NM_", alpha_text, "error_var", str(d), "d.npy"]), mean_err_var) #SAVE THE ERRORS
            np.save("".join(["NM_", alpha_text, "error_M_var", str(d), "d.npy"]), mean_err_M_var) #SAVE THE ERRORS
            

            np.save("".join(["NM_", alpha_text, "L2_error_", str(d), "d.npy"]), mean_L2_err) #SAVE THE ERRORS
            np.save("".join(["NM_", alpha_text, "L2_error_var", str(d), "d.npy"]), mean_L2_err_var) #SAVE THE ERRORS
            np.save("".join(["NM_", alpha_text, "L2_error_M_var", str(d), "d.npy"]), mean_L2_err_M_var) #SAVE THE ERRORS

            np.save("".join(["NM_", alpha_text, "sum_c", str(d), "d.npy"]), mean_sum_c)
            ##################################
    

# Reproduce plots from paper

In [ ]:
d = 6
alpha = 4

#number of workers for parallel processing
num_workers = 7

#number of times to compute the error for each size
num_intervals = 20

#constant for sampling
samp_const = 1.2  #(should be > f_max) 
# 1.1 for standard f

#values to iterate over
size = np.arange(1,7+1) # M = 10**size is the number of samples
K_size = 7 # K = 10**K_size is the number of samples used to compute the kernel estimate
num_points =  np.array([5,7,11]) #number of lattice points

lams = np.array([0.8, 0.4, 0.1, 0.01]) #lambda for the kernel
lam_texts = ["8", "4", "1", "01"] #lambda for the kernel (for saving)


#vector for storing the errors (means)
mean_err = np.zeros((len(num_points), len(size))).T
mean_err_var = np.zeros((len(num_points), len(size))).T
mean_err_M_var = np.zeros((len(num_points), len(size))).T
mean_L2_err = np.zeros((len(num_points), len(size))).T
mean_L2_err_var = np.zeros((len(num_points), len(size))).T
mean_L2_err_M_var = np.zeros((len(num_points), len(size))).T


#lattice for L2 error
Y = gen_samples(f, int(1e2), d)
approx = KernelApprox(Y, 0.01, n = int(1e4), alpha = alpha, num_workers = num_workers)
approx.gen_lattice()
lattice = approx.lattice

for l in tqdm(range(len(lams))):
    lam = lams[l]
    lam_text = lam_texts[l]
    if alpha != 4:
        lam_text = "".join([lam_text, "_", str(alpha)])

    for j in tqdm(range(len(size))): #iterate over the number of samples (M)
        print('='*80) #for clarity (splits iterations)
        print("M = 10^", size[j])
        M = int(10**size[j]) #number of samples

        #adjust based on M to get good estimates of the error
        num_intervals_adjusted = num_intervals #* max( 5 - j , 1)

        #vector for storing the errors
        err = np.zeros((len(num_points), num_intervals_adjusted))
        err_var = np.zeros((len(num_points), num_intervals_adjusted))
        L2_err = np.zeros((len(num_points), num_intervals_adjusted))
        L2_err_var = np.zeros((len(num_points), num_intervals_adjusted))

        for num in tqdm(range(num_intervals_adjusted)): #iterate over the number of times to compute the error for each size

            #generate the samples
            Y = gen_samples(f, M, d)

            for i in tqdm(range(len(num_points)), leave = False, disable=True): #iterate over the number of lattice points (n)
            
                #####################################
                    
                #compute the kernel estimate
                approx = KernelApprox(Y, lam, n = num_points[i], alpha = alpha, num_workers = num_workers, verbose=False)
                approx.gen_c()

                np.save("".join(["c_", str(d), "d.npy"]), approx.c) #SAVE THE C VALUES

                ##################################
                K = int(10**K_size) #number of samples used to compute the kernel estimate (K)
                
                err[i, num] = approx.calc_error(f, g, K, num_workers = num_workers, const = samp_const)
                err_var[i, num] = approx.error_var

                L2_err[i, num] = approx.calc_error_L2(f, N = int(1e4), x = lattice)
                L2_err_var[i, num] = approx.L2_error_var
                
                mean_err[j] = np.mean(  ( err ), axis = 1)
                mean_err_var[j] = np.mean(  ( err_var ), axis = 1)
                mean_err_M_var[j] = np.var(  ( err ), axis = 1)
                mean_L2_err[j] = np.mean(  ( L2_err ), axis = 1)
                mean_L2_err_var[j] = np.mean(  ( L2_err_var ), axis = 1)
                mean_L2_err_M_var[j] = np.var(  ( L2_err ), axis = 1)

            np.save("".join(["NM_", lam_text,"_error_", str(d), "d.npy"]), mean_err) #SAVE THE ERRORS
            np.save("".join(["NM_", lam_text,"_error_var", str(d), "d.npy"]), mean_err_var) #SAVE THE ERRORS
            np.save("".join(["NM_", lam_text,"_error_M_var", str(d), "d.npy"]), mean_err_M_var) #SAVE THE ERRORS
                

            np.save("".join(["NM_", lam_text,"_L2_error_", str(d), "d.npy"]), mean_L2_err) #SAVE THE ERRORS
            np.save("".join(["NM_", lam_text,"_L2_error_var", str(d), "d.npy"]), mean_L2_err_var) #SAVE THE ERRORS
            np.save("".join(["NM_", lam_text,"_L2_error_M_var", str(d), "d.npy"]), mean_L2_err_M_var) #SAVE THE ERRORS
                ##################################



#### $\lambda$

In [ ]:
d = 6
alpha = 4

#number of workers for parallel processing
num_workers = 12

#number of times to compute the error for each size
num_intervals = 20

#constant for sampling
samp_const = 1.2  #(should be > f_max) 
# 1.1 for standard f

#values to iterate over
size =  7 # M = 10**size is the number of samples
K_size = 7 # K = 10**K_size is the number of samples used to compute the kernel estimate
num_points =  23 #number of lattice points

lams = 0.7**np.arange(70+1) #lambda for the kernel



#vector for storing the errors (means)
mean_err = np.zeros(len(lams))
mean_err_var = np.zeros(len(lams))
mean_err_M_var = np.zeros(len(lams))
mean_L2_err = np.zeros(len(lams))
mean_L2_err_var = np.zeros(len(lams))
mean_L2_err_M_var = np.zeros(len(lams))

#lattice for L2 error
Y = gen_samples(f, int(1e2), d)
approx = KernelApprox(Y, 0.01, n = int(1e4), alpha = alpha, num_workers = num_workers)
approx.gen_lattice()
lattice = approx.lattice

lam_text = ""
if alpha != 4:
    lam_text = "".join([lam_text, "_", str(alpha)])



M = int(10**size) #number of samples

#vector for storing the errors
err = np.zeros((len(lams), num_intervals))
err_var = np.zeros((len(lams), num_intervals))
L2_err = np.zeros((len(lams), num_intervals))
L2_err_var = np.zeros((len(lams), num_intervals))

for num in tqdm(range(num_intervals)): #iterate over the number of times to compute the error for each size

    #generate the samples
    Y = gen_samples(f, M, d)

    for i in tqdm(range(len(lams)), leave = False, disable=False): #iterate over the number of lattice points (n)
                #####################################
                    
        #compute the kernel estimate
        approx = KernelApprox(Y, lams[i], n = num_points, alpha = alpha, num_workers = num_workers, verbose=False)
        approx.gen_c()

                ##################################
        K = int(10**K_size) #number of samples used to compute the kernel estimate (K)
                
        err[i, num] = approx.calc_error(f, g, K, num_workers = num_workers, const = samp_const)
        err_var[i, num] = approx.error_var

        L2_err[i, num] = approx.calc_error_L2(f, N = int(1e4), x = lattice)
        L2_err_var[i, num] = approx.L2_error_var
                
        mean_err = np.mean(  ( err ), axis = 1)
        mean_err_var = np.mean(  ( err_var ), axis = 1)
        mean_err_M_var = np.var(  ( err ), axis = 1)
        mean_L2_err = np.mean(  ( L2_err ), axis = 1)
        mean_L2_err_var = np.mean(  ( L2_err_var ), axis = 1)
        mean_L2_err_M_var = np.var(  ( L2_err ), axis = 1)

    np.save("".join(["lam_", lam_text,"_error_", str(d), "d.npy"]), mean_err) #SAVE THE ERRORS
    np.save("".join(["lam_", lam_text,"_error_var", str(d), "d.npy"]), mean_err_var) #SAVE THE ERRORS
    np.save("".join(["lam_", lam_text,"_error_M_var", str(d), "d.npy"]), mean_err_M_var) #SAVE THE ERRORS
                

    np.save("".join(["lam_", lam_text,"_L2_error_", str(d), "d.npy"]), mean_L2_err) #SAVE THE ERRORS
    np.save("".join(["lam_", lam_text,"_L2_error_var", str(d), "d.npy"]), mean_L2_err_var) #SAVE THE ERRORS
    np.save("".join(["lam_", lam_text,"_L2_error_M_var", str(d), "d.npy"]), mean_L2_err_M_var) #SAVE THE ERRORS
                ##################################
    

#### Auto selection of $\lambda$ and $n$

In [ ]:
d = 6
alpha = 4

#number of workers for parallel processing
num_workers = 12

#number of times to compute the error for each size
num_intervals = 20

#constant for sampling
samp_const = 1.2  #(should be > f_max) 
# 1.1 for standard f

#values to iterate over
size =  np.arange(1, 7+1) # M = 10**size is the number of samples
K_size = 6 # K = 10**K_size is the number of samples used to compute the kernel estimate




#vector for storing the errors (means)
mean_err = np.zeros(len(size))
mean_err_var = np.zeros(len(size))
mean_err_M_var = np.zeros(len(size))
mean_L2_err = np.zeros(len(size))
mean_L2_err_var = np.zeros(len(size))
mean_L2_err_M_var = np.zeros(len(size))

#get true integral
#Y = gen_samples(f, int(1e8), d)
#int_gf = np.mean(g(Y))
#print("True integral:", int_gf, "+/-", np.std(g(Y))/np.sqrt(1e8)) #print the true integral
#int_gf = 0.08794891440769671

#lattice for L2 error
Y = gen_samples(f, int(1e2), d)
approx = KernelApprox(Y, 0.01, n = int(1e4), alpha = alpha, num_workers = num_workers)
approx.gen_lattice()
lattice = approx.lattice

lam_text = ""
if alpha != 4:
    lam_text = "".join([lam_text, "_", str(alpha)])





#vector for storing the errors
err = np.zeros((len(size), num_intervals))
err_var = np.zeros((len(size), num_intervals))
L2_err = np.zeros((len(size), num_intervals))
L2_err_var = np.zeros((len(size), num_intervals))

for num in tqdm(range(num_intervals)): #iterate over the number of times to compute the error for each size


    for i in tqdm(range(len(size)), leave = False, disable=False): #iterate over the number of lattice points (n)
                #####################################
        
        M = int(10**size[i]) #number of samples
        #generate the samples
        Y = gen_samples(f, M, d)

        #compute the kernel estimate
        approx = KernelApprox(Y, alpha = alpha, num_workers = num_workers, verbose=False)
        approx.gen_c()

        #print("M:", M, "; N:", approx.n, "; lam:", approx.lamda)

                ##################################
        K = int(10**K_size) #number of samples used to compute the kernel estimate (K)
                
        err[i, num] = approx.calc_error(f, g, K, num_workers = num_workers, const = samp_const)
        err_var[i, num] = approx.error_var

        L2_err[i, num] = approx.calc_error_L2(f, N = int(1e4), x = lattice)
        L2_err_var[i, num] = approx.L2_error_var
                
        mean_err = np.mean(  ( err ), axis = 1)
        mean_err_var = np.mean(  ( err_var ), axis = 1)
        mean_err_M_var = np.var(  ( err ), axis = 1)
        mean_L2_err = np.mean(  ( L2_err ), axis = 1)
        mean_L2_err_var = np.mean(  ( L2_err_var ), axis = 1)
        mean_L2_err_M_var = np.var(  ( L2_err ), axis = 1)

    np.save("".join(["M", lam_text,"_error_", str(d), "d.npy"]), mean_err) #SAVE THE ERRORS
    np.save("".join(["M", lam_text,"_error_var", str(d), "d.npy"]), mean_err_var) #SAVE THE ERRORS
    np.save("".join(["M", lam_text,"_error_M_var", str(d), "d.npy"]), mean_err_M_var) #SAVE THE ERRORS
                

    np.save("".join(["M", lam_text,"_L2_error_", str(d), "d.npy"]), mean_L2_err) #SAVE THE ERRORS
    np.save("".join(["M", lam_text,"_L2_error_var", str(d), "d.npy"]), mean_L2_err_var) #SAVE THE ERRORS
    np.save("".join(["M", lam_text,"_L2_error_M_var", str(d), "d.npy"]), mean_L2_err_M_var) #SAVE THE ERRORS
                ##################################


                #print('-'*80) #dashes for clarity
            #print('*'*80) #asterisks for clarity (splits iterations)
        #print('='*80) #for clarity (splits iterations)
    